<a href="https://colab.research.google.com/github/majidraza1228/LLM/blob/LLM_Tech_Class/RAG_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')


In [ ]:
!pip install llama-index faiss-cpu pandas python-dotenv openai transformers numpy
!pip install llama-index-agent-openai llama-index-cli llama-index-core llama-index-embeddings-openai
!pip install llama-index-llms-openai llama-index-program-openai llama-index-question-gen-openai llama-index-readers-file
!pip install llama-index-readers-llama-parse llama-index-vector-stores-faiss llama-parse llama-index-indices-managed-llama-cloud

from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import PagedCSVReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
import faiss
import os
import pandas as pd

from google.colab import userdata

In [3]:
EMBED_DIMENSION=512
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=EMBED_DIMENSION)

file_path = ('/content/test.csv')
data = pd.read_csv(file_path)
data.head()

fais_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=fais_index)

In [ ]:
csv_reader = PagedCSVReader()

reader = SimpleDirectoryReader(
    input_files=[file_path],
    file_extractor= {".csv": csv_reader}
    )

docs = reader.load_data()

print(docs[0].text)

pipeline = IngestionPipeline(
    vector_store=vector_store,
    documents=docs
)

nodes = pipeline.run()

vector_store_index = VectorStoreIndex(nodes)

In [ ]:
query_engine = vector_store_index.as_query_engine(similarity_top_k=2)

response = query_engine.query("Tell me about restaurants?")
response.response